In [7]:
from langgraph.graph import StateGraph, START, END

import os
from langchain.chat_models.base import init_chat_model
from states import State
from langgraph.checkpoint.redis import RedisSaver
from states import *
from prompts import *
import dotenv
dotenv.load_dotenv()

from parsers import parse_directory_files, parse_pdf, parse_txt, parse_excel, parse_csv, parse_image_with_vision

from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, ToolMessage


In [ ]:
# h = parse_image_with_vision("/Users/omar/work/andrea/data/employee_payroll.png")

22:19:49 httpx INFO   HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="o4-mini", reasoning_effort="high")
# state = State(data_path="/Users/omar/work/andrea/data")

files = parse_directory_files("/Users/omar/work/andrea/data_temp")



    

22:28:00 httpx INFO   HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


In [12]:
files = parse_directory_files("/Users/omar/work/andrea/data")
l = []
for file in files:
    messages = [
        SystemMessage(content=METADATA_EXTRACTOR_PROMPT),
        HumanMessage(content="File Name: " + file['file_name'] + "\nFile Content: " + file['content'])
    ]


    response = model.with_structured_output(DocMetadata).invoke(messages)
    l.append(response)
    print(response)
    print("-"*100)

22:31:59 httpx INFO   HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
22:32:13 httpx INFO   HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
22:32:20 httpx INFO   HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
22:32:21 httpx INFO   HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
22:32:27 httpx INFO   HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
22:32:36 httpx INFO   HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
name='employee_payroll.png' path='employee_payroll.png' purpose="Document ACME Inc.'s payroll register for the bi-weekly pay period 7/20/2024 – 08/02/2024, detailing employee gross pay, taxable amounts, net pay, and payment status." possible_use_cases=['Payroll processing and verification', 'Financial auditing and reporting', 'Tax filing and compliance', 'Employee compensation analysis', 'Record keeping and archival']
-

In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="o4-mini", reasoning_effort="high")


def metadata_extractor(state: State):
    files = parse_directory_files(state['data'])

    for file in files:

        messages = [
            SystemMessage(content=METADATA_EXTRACTOR_PROMPT),
            HumanMessage(content="File Name: " + file['file_name'] + "\nFile Content: " + file['content'])
        ]


    response = model.with_structured_output(DocMetadata).invoke(messages)
    state['doc_metadata'].append(response)
    return state

def document_classifier(state: State):
    return state

def task_manager(state: State):
    return state

def doc_selector_agent(state: State):
    return state

def execution_agent(state: State):
    return state

def reflector(state: State):
    return state

def reporter(state: State):
    return state

def should_continue(state: State):
    return state


In [ ]:


builder = StateGraph(State)
builder.add_node("metadata_extractor", metadata_extractor)
builder.add_node("document_classifier", document_classifier)
builder.add_node("task_manager", task_manager)
builder.add_node("doc_selector_agent", doc_selector_agent)
builder.add_node("execution_agent", execution_agent)
builder.add_node("reflector", reflector)
builder.add_node("reporter", reporter)

builder.add_edge(START, "metadata_extractor")
builder.add_edge("metadata_extractor", "document_classifier")
builder.add_edge("document_classifier", "task_manager")
builder.add_edge("task_manager", "doc_selector_agent")
builder.add_edge("doc_selector_agent", "execution_agent")
builder.add_edge("execution_agent", "reflector")

builder.add_conditional_edges("reflector",should_continue, {"pass": "reporter",
    "fail": "execution_agent"})

# builder.add_edge("reflector", "reporter")
builder.add_edge("reporter", END)

REDIS_URI = "redis://localhost:6379/0"
with RedisSaver.from_conn_string(REDIS_URI) as checkpointer:
    checkpointer.setup()
    
    graph = builder.compile(checkpointer=checkpointer)
